# Homework 3 - Find the perfect place to stay in Texas!

In [1]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import numpy as np
import scipy
import heapq
from tqdm import tqdm

In [2]:
with open('Airbnb_Texas_Rentals.csv','r',encoding="utf8") as csv_file: #opening the csv file in read mode
    csv_reader=csv.reader(csv_file)  #csv reader allows us to iterate through its lines
    
    with open('Airbnb_Texas_Rentals.tsv','w',encoding="utf8") as new_file:  #opening a new tsv file (still empty)
        csv_writer=csv.writer(new_file,delimiter='\t')  #writer allows us to write rows, and we separate the fields in these
                                                        #rows with the tab delimiter
        for line in csv_reader: #we iterate through the reader's lines 
               csv_writer.writerow(line[1:]) #write those lines (except the first element that is the index) on the new tsv

with open('Airbnb_Texas_Rentals.tsv','r',encoding="utf8") as all_files: #now we want all the documents so we first open the big tsv
    for i, row in enumerate(all_files):  #iterating through the lanes and keeping track of the order of the operation
        if i > 0:   #skipping the first line (the header)
            if row != '\n': #the rows were separated by blank spaces in the form of '\n' so we skip them
                with open('doc\doc_' + str(int((i-(i/2)))) + '.tsv','w',encoding="utf8") as new_files: #opening a new tsv for every line, (the str(int((i-(i/2))) is to keep the number of the docs 0,1,2 ecc instead of 0, 2, 4...)
                    new_files.write(row) #writing

In [3]:
def save_dict_to_file(dic, file): #defining a function to save files right away
    f = open('{}.txt'.format(file), 'w',encoding="utf8") #open a new file in write mode (empty)
    f.write(str(dic)) #write in it what we need (it will always be a dictionary, hence 'dic')
    f.close() #closing it

In [5]:
docpaths = [f for f in listdir("doc") if isfile(join("doc", f))] #joining the paths for the tsv documents in the directory

documentlist = {} #this will be for keeping track of the words in the docs {doc_i : ['house, 'garden'.....]}
vocabulary = {} #this will be useful to keep track of the word and its id {'house':1, 'garden':2}
inverted_index = {} #this is the inv idx: {1: [doc_1, doc_2, ....], 2: [doc_2, doc_6, ....]}
indexOfWord = 0 #this is used to give the id to the words in the vocabulary
for doc in docpaths: #for every document in the list of documents...
    with open('doc\{}'.format(doc), 'r',encoding="utf8") as file: #open the doc in read mode
        lines = file.readlines() #read its lines
        if len(lines) > 0 : #leaving out all the empty lines
            document = lines[0] #its a list of only one element, so we take that as a string
            cols = document.replace('\\n',' ').split('\t') #cleaning
            
            to_tokenize = cols[4] + cols[7] #taking description and title to tokenize
            tokens = nltk.word_tokenize(to_tokenize) #tokenization
            filtered_words = [nltk.stem.PorterStemmer().stem(word) for word in tokens if word not in nltk.corpus.stopwords.words('english') and word not in ".,'()"]
            # removing stopwords, special characters, stemming
            documentlist[doc] = filtered_words #creating a document list: for every document we will have the tokenized words in title and description
            
            for w in filtered_words: 
                if w not in vocabulary: #creating the vocabulary
                    indexOfWord += 1    #assigning an index that ranges from 1 to whatever
                    vocabulary[w] = indexOfWord  #for every index we give the word
                
                if vocabulary[w] not in inverted_index: #creating the inverted index
                    temp = []  #creating a list so we can append the documents that have that word
                    temp.append(doc)  #appending the first doc
                    inverted_index[vocabulary[w]] = temp #creating the index key and giving them the value
                elif doc not in inverted_index[vocabulary[w]]: #if the key was already created
                    inverted_index[vocabulary[w]].append(doc)  

#saving the files
save_dict_to_file(inverted_index,"inverted_index")
save_dict_to_file(vocabulary,"vocabulary")
save_dict_to_file(documentlist,"documentlist")

100%|████████████████████████████████████████████████████████████████████████████| 18259/18259 [07:32<00:00, 40.39it/s]


In [7]:
len(docpaths)

18259

In [24]:
query = input() #taking the user query
tokens = nltk.word_tokenize(query) #tokenizing the query
query_tokens = [nltk.stem.PorterStemmer().stem(token) for token in tokens if token not in stopwords.words('english') if token not in ".,'()"]
#removing stopwords, special characters, stemming

vocabulary = open('vocabulary.txt', 'r', encoding = 'utf-8') #opening the vocabulary in read mode
vocabulary = eval(vocabulary.read()) #if we dont do eval the file will be considered a string. eval 'understands' what type of object it is

inverted_index = open('inverted_index.txt', 'r', encoding = 'utf-8')  #same here
inverted_index = eval(inverted_index.read())
        
term_ids = [vocabulary[token] for token in query_tokens if token in vocabulary] #taking the ids of the query tokenized words
search_results = [inverted_index[i] for i in term_ids if i in inverted_index] #taking the documents in the inv idx for that id

new_list = [set(list_) for list_ in search_results] #removing duplicates and preparing for intersection
intersect = set.intersection(*new_list) #gives us the documents that have all the words of the query

col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url"]
#making a list for the column names
df = pd.DataFrame(columns = col_names) #creating an empty df with the list cerated before

for j, doc in enumerate(intersect): #iterating through the intersection list and keeping track of the order
    with open(r'doc\{}'.format(doc), 'r',encoding="utf-8") as file: #opening the files in read mode
        csv_reader = csv.reader(file, delimiter = '\t') #csv.reader to iterate
        
        for line in csv_reader: #iterating through the lines. 1 line = the whole document
            for i, field in enumerate(line): #field = tab separated field
                df.loc[j, col_names[i]] = field #giving to [row j ,col i] the value of the field
        
pd.set_option('max_colwidth',500) #making the rows more readabale(bigger)
display(df[['title', 'description', 'city', 'url']].style.set_table_styles([ {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}]))
#displaying the df without the index

roof dog


,title,description,city,url
0,Cozy furnished home on the EastSide!,"Welcome to our home!\n\nAn eclectic combination of hip and modern all under one roof.\n\nCool Amenities like Retro Lounge, Super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations, yoga studio, and fitness center.\n\nThe Metro Train is connected to the apartment building and will take you directly to downtown in 8 minutes or less depending on your drop off location. So convenient, no need for a car or anything!",Austin,https://www.airbnb.com/rooms/18795604?location=Cedar%20Park%2C%20TX
1,Palm Oaks Ranch 1521RP,"Welcome to Palms Oaks Ranch! This gorgeous two-story 4 bedroom/3 bath Mediterranean style home on 18 acres over looks a private lake. This property is surrounded by live oak trees with horse corrals, riding arena, stables, tack room and an ac dog run. The exterior of the home is stucco with a spanish style roof. Flooring is satillo tile with marble in the bathrooms. The kitchen features a gas cook top, dishwasher, stainless teal appliances and granite counter tops. The open floor plan features cedar ceilings and a fabulous staircase. Several large arch windows allow for plenty of light. Three of the bedrooms are located upstairs, with the master having an ensuite, king bed and a balcony overlooking the lake. The second bedroom has a king and the third a queen. The fourth bedroom is located downstairs and also has a king size bed. This property offers everything you need for a relaxing weekend away from it all. Roam the 18 acres watching the wildlife or fish into the sunset in the lake. This home is located just a 3 minute drive to Paradise Key. Paradise Key offers a double boat dock, Valet boat barn and a dockside bar and grill. This is a great spot for Weddings, special occasion parties or a weekend away. Wedding venue fee is $500. For a virtual tour click here",Rockport,https://www.airbnb.com/rooms/12009441?location=Bayside%2C%20TX
2,Furnished EastSide Flat with a Spacious Balcony.,"Welcome to our home.\n\nLocated on the East Side of Austin just steps from Austin’s newest rail stop, Platform is an eclectic combination of hip and modern all under one roof. \n\nComplete with a Retro Game Lounge, super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations for sizzling social hours, yoga studio, fitness and conditioning center, and so much more. \n\nGet in your groove and get booking.",Austin,https://www.airbnb.com/rooms/18123352?location=Cedar%20Park%2C%20TX
3,Comfy Flat on the EastSide + a Balcony & by Metro!,"Welcome :)\n\nAn eclectic combination of hip and modern all under one roof.\n\nCool Amenities like Retro Lounge, Super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations, yoga studio, and fitness center.\n\nThe Metro Train is connected to the apartment building and will take you directly to downtown in 8 minutes or less depending on your drop off location. So convenient, no need for a car or anything!",Austin,https://www.airbnb.com/rooms/18795759?location=Cedar%20Park%2C%20TX


In [ ]:
term_ids = [vocabulary[token] for token in query_tokens if token in vocabulary]

In [8]:
#TF = how may times the word is repeated within the i-th document divided by its length (term frequency)
def tf(word, document):
    return document.count(word) / len(document) 

#this is to have the number of times the word is repeated in one document
def document_frequency(word):
    if word in vocabulary:
        term_id = vocabulary[word]
    return len(inverted_index[term_id])

#IDF(word) = loge(Total Number Of Documents / Number Of Documents with term 'word' in it)
def idf(word):
    return math.log(len(docpaths) / document_frequency(word))

#TFIDF = TF * IDF
def tfidf(word, document):
    return tf(word, document) * idf(word)

In [10]:
vocabulary = open('vocabulary.txt', 'r', encoding = 'utf-8')
vocabulary = eval(vocabulary.read()) 

inverted_index = open('inverted_index.txt', 'r', encoding = 'utf-8') 
inverted_index = eval(inverted_index.read())

document_list = open('documentlist.txt', 'r', encoding = 'utf-8')
file = eval(document_list.read())

new_inverted_index = {}
for key,doc in tqdm(documentlist.items()): #taking the keys (doc_i) and the values 
    for w in doc:#w = word in the doc, doc = ['house', 'garden', ....]
        score = tfidf(w,doc) #tf idf of the word in the doc
        w_index = (key,score) #tuple of key (doc_i), and score (tf/idf)
        if vocabulary[w] not in new_inverted_index: #if the id of the word is not in the new inv_idx
            temp = [] #initializing empty list
            temp.append(w_index) #appending the tuple
            new_inverted_index[vocabulary[w]] = temp #giving to the newly created key (that is the id), the list just created [(doc_i, tf/idf), (doc_i+1, tf/idf)]
        elif doc not in new_inverted_index[vocabulary[w]]: #if the doc is not already in that list: 
            new_inverted_index[vocabulary[w]].append(w_index) #append to the list the new tuple

#removing duplicates
for key in new_inverted_index:
    new_inverted_index[key] = list(set(new_inverted_index[key]))
#saving
save_dict_to_file(new_inverted_index,"inverted_index_tfidf")

100%|███████████████████████████████████████████████████████████████████████████| 18259/18259 [00:38<00:00, 473.71it/s]


In [15]:
query = input() #input query
tokens = nltk.word_tokenize(query) #tokenization
query_tokens = [nltk.stem.PorterStemmer().stem(token) for token in tokens if token not in stopwords.words('english') if token not in ".,'()"]
#removing stopwords, special characters, stemming

tfidf_query_array = [] 
for w in query_tokens: #scores 
    score = 1/math.sqrt(len(query_tokens)) #score for the words the 1/math.sqrt(len(query_tokens)) is to normalize the scores
    tfidf_query_array.append(score) #appending to a list for new score
#print(tfidf_query_array)

roof dog


In [16]:
inverted_index_tfidf = open("inverted_index_tfidf.txt", 'r', encoding = 'utf-8') #opening the new invd_idx and evaluating 
inverted_index_tfidf = eval(inverted_index_tfidf.read())


term_ids = [vocabulary[token] for token in query_tokens if token in vocabulary] #terms ids of the words of the query
search_results = [inverted_index_tfidf[idx] for idx in term_ids if idx in inverted_index_tfidf] #list of tuples (doc_i, score) for that id

cos_arrays = {} #dictionary for the cosine sim values for the docs
for list_ in search_results: #for every list of tuple (doc_i, score) for every id
    for tuple_ in list_: #for every tuple:
        if tuple_[0] not in cos_arrays: #if the doc is not in that tuple
            temp = []
            temp.append(tuple_[1]) #append to the newly created list the tf/idf
            cos_arrays[tuple_[0]] = temp #create the key (doc_i) and give the values as the list of tf/idfs
        else:
            cos_arrays[tuple_[0]].append(tuple_[1]) #if the key is already there append the value to the list

final = [(key,cos_arrays[key]) for key in cos_arrays if len(cos_arrays[key]) == len(term_ids)] #take the documents that have all of the words
print(final)

[('doc_7428.tsv', [0.11172773927519879, 0.06463152033532973]), ('doc_8919.tsv', [0.09969552119940815, 0.05767120276075577]), ('doc_14141.tsv', [0.04050130548725956, 0.02342892612155703]), ('doc_6904.tsv', [0.10286045838034173, 0.05950203459443055])]


In [17]:
doc_sim = {} #dic for {doc_i : similarity_score, ...}
for tuple_ in final: #take the tuple in final (the values are the list of the tf/idf scores)
    sim = 1 - (scipy.spatial.distance.cosine(np.array(tfidf_query_array) , np.array(tuple_[1]))) #use this to compute the the cos similarity
    doc_sim[tuple_[0]] =  sim #key = doc_i, value = similarity
print(doc_sim)

{'doc_7428.tsv': 0.9661433851761906, 'doc_8919.tsv': 0.9661433851761907, 'doc_14141.tsv': 0.9661433851761906, 'doc_6904.tsv': 0.9661433851761907}


In [18]:
heap = [(-value, key) for key, value in doc_sim.items()] #list that has a tuple ordered by the smallest to the largest
largest = heapq.nsmallest(10, heap) #taking the smallest (they will actually be the largest)
largest = [(key, -value) for value, key in largest] #re-ordering keys and values
print(largest)

col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url", "similarity"]
df = pd.DataFrame(columns = col_names)#same as before

for j, doc in enumerate(largest): 
    with open(r'doc\{}'.format(doc[0]), 'r',encoding="utf-8") as file:
        
        csv_reader = csv.reader(file, delimiter = '\t')
        for line in csv_reader:
            for i, field in enumerate(line):
                df.loc[j, col_names[i]] = field
                df.loc[j, 'similarity'] = doc[1]
        
pd.set_option('max_colwidth',500)
display(df[['title', 'description', 'city', 'url', 'similarity']].style.set_table_styles([ {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}]))

[('doc_6904.tsv', 0.9661433851761907), ('doc_8919.tsv', 0.9661433851761907), ('doc_14141.tsv', 0.9661433851761906), ('doc_7428.tsv', 0.9661433851761906)]


,title,description,city,url,similarity
0,Cozy furnished home on the EastSide!,"Welcome to our home!\n\nAn eclectic combination of hip and modern all under one roof.\n\nCool Amenities like Retro Lounge, Super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations, yoga studio, and fitness center.\n\nThe Metro Train is connected to the apartment building and will take you directly to downtown in 8 minutes or less depending on your drop off location. So convenient, no need for a car or anything!",Austin,https://www.airbnb.com/rooms/18795604?location=Cedar%20Park%2C%20TX,0.966143
1,Comfy Flat on the EastSide + a Balcony & by Metro!,"Welcome :)\n\nAn eclectic combination of hip and modern all under one roof.\n\nCool Amenities like Retro Lounge, Super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations, yoga studio, and fitness center.\n\nThe Metro Train is connected to the apartment building and will take you directly to downtown in 8 minutes or less depending on your drop off location. So convenient, no need for a car or anything!",Austin,https://www.airbnb.com/rooms/18795759?location=Cedar%20Park%2C%20TX,0.966143
2,Palm Oaks Ranch 1521RP,"Welcome to Palms Oaks Ranch! This gorgeous two-story 4 bedroom/3 bath Mediterranean style home on 18 acres over looks a private lake. This property is surrounded by live oak trees with horse corrals, riding arena, stables, tack room and an ac dog run. The exterior of the home is stucco with a spanish style roof. Flooring is satillo tile with marble in the bathrooms. The kitchen features a gas cook top, dishwasher, stainless teal appliances and granite counter tops. The open floor plan features cedar ceilings and a fabulous staircase. Several large arch windows allow for plenty of light. Three of the bedrooms are located upstairs, with the master having an ensuite, king bed and a balcony overlooking the lake. The second bedroom has a king and the third a queen. The fourth bedroom is located downstairs and also has a king size bed. This property offers everything you need for a relaxing weekend away from it all. Roam the 18 acres watching the wildlife or fish into the sunset in the lake. This home is located just a 3 minute drive to Paradise Key. Paradise Key offers a double boat dock, Valet boat barn and a dockside bar and grill. This is a great spot for Weddings, special occasion parties or a weekend away. Wedding venue fee is $500. For a virtual tour click here",Rockport,https://www.airbnb.com/rooms/12009441?location=Bayside%2C%20TX,0.966143
3,Furnished EastSide Flat with a Spacious Balcony.,"Welcome to our home.\n\nLocated on the East Side of Austin just steps from Austin’s newest rail stop, Platform is an eclectic combination of hip and modern all under one roof. \n\nComplete with a Retro Game Lounge, super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations for sizzling social hours, yoga studio, fitness and conditioning center, and so much more. \n\nGet in your groove and get booking.",Austin,https://www.airbnb.com/rooms/18123352?location=Cedar%20Park%2C%20TX,0.966143


# step 4

In [19]:
df4 = pd.read_csv('Airbnb_Texas_Rentals.csv') #creating dataframe from the csv
city_lat_lon = df4.groupby('city').mean()[['latitude','longitude']] #grouping by the city and taking the mean.

In [21]:
user_query_city = input('Enter city: ') #asking the user for the city
user_query_price = input('Enter your preferred daily price: ') #asking the user for the price
user_query_bedrooms = input('Enter your preferred number of bedrooms: ') #asking the user for the bedrooms

Enter city: San Antonio
Enter your preferred daily price: 40
Enter your preferred number of bedrooms: 2


In [73]:
def score(doc_price ,doc_bedrooms, doc_latitude, doc_longitude): #defining the new score
    global user_query_city, user_query_price , user_query_bedrooms, city_lat_lon #calling the global variables inside the function
    
    score = 0 #score default
    
    city = user_query_city #defining the new variable city
    price = int(user_query_price .replace('$', '')) #if in the price the user uses a dollar, we take it out
    bedrooms = int(user_query_bedrooms) #just the number of bedrooms
    
    
    
    if city in city_lat_lon.index: #if the user's city is in the index of the groupby
        lat_lon = list(city_lat_lon.loc[city]) #take the long and lat of the row with the name
        lat_lon_diff = [float(lat_lon[0]) - float(doc_latitude), float(lat_lon[1]) - float(doc_longitude)] #diffrence between longitude and latitude

        if (doc_price[1:].isdigit() == True) and (doc_bedrooms.isdigit() == True): #if price and bedrooms are actual numbers:
            if (abs(lat_lon_diff[0]) < 1.5) and (abs(lat_lon_diff[1]) < 1.5): #if the difference is in a max range of 1.5 has full score
                score = (1/(1 + (abs(float(doc_price[1:]) - price))) * 0.6) + (1/(1 + (abs(int(doc_bedrooms) - bedrooms)))) * 0.4
            else: #if its outside the penalty is score*0.1
                score = ((1/(1 + (abs(float(doc_price[1:]) - price))) * 0.6) + (1/(1 + (abs(int(doc_bedrooms) - bedrooms)))) * 0.4) * 0.1
                #score is done using the 1/1+x, where x is the difference of prices or bedrooms. This means that if the difference is large the score will be lower
    return score
    

In [74]:
step4_idx = {}#dictionary that will map to every document the new score
for j, doc in enumerate(intersect): #taking the documents of the intersection in point 3.1
    with open(r'doc\{}'.format(doc), 'r',encoding="utf-8") as file: #opening the tsv files
        lines = file.readlines() #reading the files
        if len(lines) > 0 : 
            document = lines[0] #as before...
            cols = document.replace('\n',' ').split('\t') 
            doc_price, doc_bedrooms, doc_latitude, doc_longitude = cols[0], cols[1], cols[5], cols[6]
            new_score = score(doc_price ,doc_bedrooms, doc_latitude, doc_longitude)
            step4_idx[doc] = new_score #key = doc_i, value = new_score
            
save_dict_to_file(step4_idx,"step4_idx")
print(step4_idx)

{'doc_6904.tsv': 0.20750000000000002, 'doc_14141.tsv': 0.013450980392156864, 'doc_7428.tsv': 0.20652173913043478, 'doc_8919.tsv': 0.20759493670886078}


In [26]:
heap = [(-value, key) for key, value in step4_idx.items()] #same as before
largest = heapq.nsmallest(5, heap)
largest = [(key, -value) for value, key in largest]
print(largest)

col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url", "score"]
df = pd.DataFrame(columns = col_names)# index = False?

for j, doc in enumerate(largest):
    with open(r'doc\{}'.format(doc[0]), 'r',encoding="utf-8") as file:
        
        csv_reader = csv.reader(file, delimiter = '\t')
        for line in csv_reader:
            for i, field in enumerate(line):
                df.loc[j, col_names[i]] = field
                df.loc[j, 'score'] = doc[1]
        
pd.set_option('max_colwidth',500)
display(df[['title', 'description', 'city', 'url', 'score']].style.set_table_styles([ {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}]))

[('doc_8919.tsv', 0.20759493670886078), ('doc_6904.tsv', 0.20750000000000002), ('doc_7428.tsv', 0.20652173913043478), ('doc_14141.tsv', 0.013450980392156864)]


,title,description,city,url,score
0,Comfy Flat on the EastSide + a Balcony & by Metro!,"Welcome :)\n\nAn eclectic combination of hip and modern all under one roof.\n\nCool Amenities like Retro Lounge, Super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations, yoga studio, and fitness center.\n\nThe Metro Train is connected to the apartment building and will take you directly to downtown in 8 minutes or less depending on your drop off location. So convenient, no need for a car or anything!",Austin,https://www.airbnb.com/rooms/18795759?location=Cedar%20Park%2C%20TX,0.207595
1,Cozy furnished home on the EastSide!,"Welcome to our home!\n\nAn eclectic combination of hip and modern all under one roof.\n\nCool Amenities like Retro Lounge, Super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations, yoga studio, and fitness center.\n\nThe Metro Train is connected to the apartment building and will take you directly to downtown in 8 minutes or less depending on your drop off location. So convenient, no need for a car or anything!",Austin,https://www.airbnb.com/rooms/18795604?location=Cedar%20Park%2C%20TX,0.2075
2,Furnished EastSide Flat with a Spacious Balcony.,"Welcome to our home.\n\nLocated on the East Side of Austin just steps from Austin’s newest rail stop, Platform is an eclectic combination of hip and modern all under one roof. \n\nComplete with a Retro Game Lounge, super “fly” poolscape consisting of three pools and intriguing Aquatic Wall water feature, Dog Park, outdoor grilling stations for sizzling social hours, yoga studio, fitness and conditioning center, and so much more. \n\nGet in your groove and get booking.",Austin,https://www.airbnb.com/rooms/18123352?location=Cedar%20Park%2C%20TX,0.206522
3,Palm Oaks Ranch 1521RP,"Welcome to Palms Oaks Ranch! This gorgeous two-story 4 bedroom/3 bath Mediterranean style home on 18 acres over looks a private lake. This property is surrounded by live oak trees with horse corrals, riding arena, stables, tack room and an ac dog run. The exterior of the home is stucco with a spanish style roof. Flooring is satillo tile with marble in the bathrooms. The kitchen features a gas cook top, dishwasher, stainless teal appliances and granite counter tops. The open floor plan features cedar ceilings and a fabulous staircase. Several large arch windows allow for plenty of light. Three of the bedrooms are located upstairs, with the master having an ensuite, king bed and a balcony overlooking the lake. The second bedroom has a king and the third a queen. The fourth bedroom is located downstairs and also has a king size bed. This property offers everything you need for a relaxing weekend away from it all. Roam the 18 acres watching the wildlife or fish into the sunset in the lake. This home is located just a 3 minute drive to Paradise Key. Paradise Key offers a double boat dock, Valet boat barn and a dockside bar and grill. This is a great spot for Weddings, special occasion parties or a weekend away. Wedding venue fee is $500. For a virtual tour click here",Rockport,https://www.airbnb.com/rooms/12009441?location=Bayside%2C%20TX,0.013451


In [109]:
coordinates = input('Enter Latitude and Longitude :').split(' ')
radius = float(input('Enter the radius'))
coordinates = list(map(float, coordinates))

Enter Latitude and Longitude :32.779594 -96.800768
Enter the radius2000


[50.0, 60.0]

In [110]:
import folium
import geopy
from geopy import distance

tooltip = ''
m = folium.Map(location = coordinates)

df = pd.read_csv('Airbnb_Texas_Rentals.csv', usecols = ['latitude', 'longitude'])

for i in range(len(df)):
    try:
        distance = geopy.distance.distance(coordinates, df.iloc[i]).km
        print(distance)
        if distance < radius:
            folium.Marker(df.iloc[i], popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(m)
    except:
        pass
    
folium.Circle(radius=radius, location=coordinates,
    popup='The Waterfront',
    color='crimson',
    fill=True,
).add_to(m)

m.save('map.html')

209.92478085060017
245.9097209175879
227.26710262497699
150.0977622485873
28.365782849939148
185.79192296061294
188.1757724357638
29.62548047861632
322.3785042354133
248.46808320895704
13.050456362841945
17.345030961091847
185.12548077784288
234.7128117531536
232.7988170279506
229.66896112209903
178.822798345409
342.2033463599787
217.02150287266156
26.995835613809028
213.24585850019395
156.83960682794512
216.1334806158768
217.16178608989395
35.88830759749223
222.40744636944294
214.9887144924248
183.88224371263428
226.56561080164846
21.066944764419326
227.84554217870715
16.850588263149074
180.29376179404946
118.95473377003938
227.5447690760466
181.5060210310117
180.70159263981256
36.37138142521512
4.329104887012118
129.8960177467185
225.76972609622172
175.01442876466234
222.31940726497308
170.7124096088969
181.52361189910778
30.750131832483348
176.73842032707077
237.15461686323752
152.78959194072476
244.78932757442755
22.531116991573025
250.48153564579985
209.61810320154075
252.26539838

17.112527697496237
30.009879657996553
27.895554505914188
187.10303350197296
352.4062058658973
57.774705388857605
152.6798901570024
1.7880710041471615
12.935273360400286
346.79818821832714
172.94282704029678
196.90674610108704
26.44946635474378
29.347440183695195
160.5923606575576
172.86085373177318
342.4273752416142
343.0693962386671
154.36688948915295
343.03063287244674
176.59979084534237
246.36213910102
210.29295533998513
12.50385798349434
131.87257918756447
28.532406385431575
215.5196030771736
13.597172418053404
41.46473673716317
244.05784862086645
154.72183361134196
241.4369148256071
224.64819120238406
178.8230883366226
248.39735479844762
172.35484847928728
84.13510306085888
217.0496141616468
194.20580720195568
226.17866546532622
160.86669700683984
223.3008325847381
233.30639862069006
246.75998540357554
342.48422349396435
121.50267786073844
245.0973382416875
225.50538597152996
152.01634058605194
11.9950039006161
240.74662244484153
226.49002245227828
21.435970237105156
209.328709548

48.7667768760366
240.54354502587825
24.688639426649083
215.72826514044777
36.83019786181682
37.681964459008945
346.81174991646134
87.36091041403242
20.568524726304016
187.91326777109492
176.55589043909532
15.283561934825753
16.783309917226152
24.22517541953915
228.37253415073874
224.3710657640847
212.79748180355583
225.15852910545155
151.89137001137738
30.759385405268116
189.1229398978533
35.142375531985756
239.34974428101907
185.75900546371943
149.3315669607979
9.0228781916941
246.50953981039498
15.384908743827259
222.523345377741
29.194728103650423
197.16307157806332
149.2885875132782
224.98492730481453
4.612583862173099
33.37348048319832
246.12376154623013
437.8003473906217
51.006384176573086
228.06803284924956
324.9492377623036
225.34242913443416
17.9880348387857
252.677706251998
173.5090518764993
225.5798092273037
262.5688698178073
244.31043061067155
15.219179472288838
38.98460666523944
2.9870094801225417
226.62473681915742
337.0157260970272
262.46174828340463
8.09265485516913
15.

221.72780800018464
209.38774375284956
1.6956659142268775
322.54482497939557
229.66096260243893
244.02573798985378
225.84742458039491
246.17898024906032
17.720459355127286
217.58989452459994
33.95192462449271
175.6561656272469
36.41036616297219
268.4835865956181
89.0298371927234
179.2814831377111
188.04328868692653
225.9217480866665
262.60047829368125
225.35850760518392
267.86894365649727
245.4328717190119
225.8423355624256
15.185219752124205
171.93811810025264
328.75090659733223
268.87572132534245
172.54814249684662
357.21927880640413
12.935273360400286
205.9685765930213
232.19779492301188
182.1599655781522
223.70514523274414
232.2326110851773
185.71087598251646
37.24937227898017
247.37387683914156
267.641687691387
357.25230236569973
218.05282415613485
238.74629438172133
185.9960872682547
269.21090229724166
186.40950865908803
223.09313579333747
57.20160111131461
38.35916437083714
242.51573457563956
191.71658510030656
268.14273845608045
245.7438658733515
212.18250413493118
246.604555939

181.47995611077292
30.353058437516285
15.569916778340712
183.71928813715618
1.078389118323825
180.88957277485966
1.1474600709619491
179.42133463185206
241.20965075405957
243.4497043434692
269.1590492932247
208.6417629781804
268.906702497613
182.2889324975382
245.36123316043935
24.734562629281488
357.10800173103405
184.5673525440502
247.4140273304608
185.77901640234114
19.712521222000284
222.2792972598481
180.34633809556394
36.5604449052925
356.7034355768472
194.76963870277254
12.210885132542133
186.62595469248063
12.236043665602734
292.1530704101441
182.6190314742488
186.35021517959612
234.69656301289527
170.49251883506776
164.07959506411197
177.02533594372431
237.5051834393755
2.8177184540103464
223.55740943692683
225.07580145213782
226.92359806143327
221.17526220854168
244.65178078241283
343.1850519835529
245.35679385874445
22.487513276173523
225.17503881293007
268.4504951858154
56.14218627078859
183.68178135527012
57.17662101076327
278.20584510438846
269.6519132509999
57.30872457081

...

250.06846652029176
5.8680835821900486
249.79248290399093
251.31849942411748
224.93825509754728
6.352136729107249
36.17825351723798
356.00641311988716
24.657980283580738
7.818385137920266
181.23704085737936
246.63764368075667
15.333233177001043
20.65297686379016
357.55851944487904
32.74000629341555
327.34145802721736
232.99509702779758
22.65169598152745
12.772583524140295
249.88259458012664
40.749492475847006
250.12780382073566
357.33990247838165
250.65888530458963
224.93601571618268
28.54645229003945
355.8438242925464
18.695042734647814
43.88298582152303
246.04590514293443
16.84633034196266
251.6925147013351
357.42806892520343
28.2328807053637
253.06497332531472
27.944418573750614
245.4171486948475
437.69498519945915
277.91171746900994
3.9871784747850905
37.35458788587801
267.91861596101745
357.4402813380214
32.602236559235344
226.70904277577137
225.34147771487324
171.1799297225013
3.184316536850305
225.60190338501835
11.427068747099872
221.50795971517397
175.67616450699077
174.9706232

51.05414532610451
34.566126938338336
249.80905989063044
173.16184509958381
351.39077770706615
173.78186361514835
322.4275012161183
227.86878098036934
349.3205766895208
180.7484372763729
225.17529541610995
33.21076855773622
2.9347155926650914
176.60878098276183
8.102659475287721
191.46914669682073
225.2114240456211
330.99613971055317
356.846176394427
1.8977840218305488
18.74407832034552
23.44843756140036
16.99554831247967
221.72780800018464
27.797813433290585
268.5169006838335
15.377342057463668
224.52896286771468
336.7442083463192
250.55080587893858
225.15959924078564
267.8789066131278
11.023497578435329
245.1113470050672
186.1705334130097
15.28804394706274
2.782451670738339
268.8456000279135
13.419316327269708
225.72553345489365
220.83334789067152
26.135179179730503
31.10080899703898
267.8282508701477
222.29594479330783
6.054000498144218
17.905809045543975
235.507938729764
243.90176626258338
225.15720704956152
245.49055329852965
34.66126857933263
9.103443912472615
57.879765122849726
2

249.73106247311907
222.95772393142335
262.9057267577758
261.4863983502437
28.58691283002583
262.83381874162285
188.3474424058379
174.1304398826915
250.59541604463251
30.951202142428365
357.2362121651641
170.54531639311975
334.98586017563093
354.861110204835
269.5022080542316
245.422973703376
4.564948357601308
183.68541338076716
18.584421060118107
268.884619728317
25.195438235181516
246.16910865414582
11.676807733029122
225.75504447850685
1.0676055445477863
329.13922869492245
249.16343935580682
27.500448606047765
8.836318270570123
31.1932579295841
1.415426635108174
217.08405762193848
225.03080282286945
329.6514459397935
28.68830823580667
0.7575424875485767
7.853642516122561
33.33092525140817
357.38924752152616
216.6728992269723
52.96565485518195
178.70997062244334
32.12748431516702
245.91414029007532
246.15794895942844
17.148646265935987
249.53514165887685
250.0464031777843
221.64838769104747
244.47840463071836
26.208499781794384
251.10562121849014
7.54049978297496
11.16310143537999
175

278.171740018649
18.668270795738408
8.994900547891543
357.9238357413249
35.51568710787372
15.695447712984114
357.38812193995415
268.80530544020206
267.4110157758643
28.05360396711222
26.208499781794384
345.09127558673276
60.497145859755356
7.967651140682435
242.9218754363211
251.78064461121272
178.25393066039163
225.5559794302929
178.94706992821685
19.17607448047213
357.4636880590451
15.155259180230168
322.51216169411236
30.87897761137556
183.91946045101838
27.913588781156015
248.57001909505493
178.82872577197946
26.36270654094806
224.83939989560196
21.72958210655162
7.660032814843498
178.80114331912446
11.302131454600334
11.216745389924272
183.16388634894926
175.85742141095548
245.59779073766643
28.283644313422197
267.9346819074156
221.06141153860565
187.52417100309134
341.51505275503257
181.60028740364888
56.76207276822309
267.48021014970146
267.4047371262828
9.47277745744326
229.2223137959167
17.92664852287928
240.0433217447676
37.65209804911892
7.7456793905306025
224.48278556107587

192.0926059772682
0.1892237732342753
4.670093166070843
262.60430185017066
268.91187538521575
246.6343436081534
36.72379761319024
244.64579998668978
33.83985315198628
330.85536008891665
221.18083868362552
187.0956363169016
29.03629132418583
324.1898006387939
171.59624596110814
27.632982433540942
11.453807055774337
234.99457542525022
1.4050891057613049
267.57171824774275
263.5045849455013
268.83336932182027
186.40950865908803
230.01794997811623
336.5759602541129
30.029504526229935
186.32241231346399
349.6643536334633
176.63615248423613
54.50980893831673
356.846176394427
357.2392486082458
2.8177184540103464
3.6862465643177833
269.5688282555213
266.0882767663409
357.9040514137884
56.48059824638824
8.624542650355984
245.4847139591678
21.46078009535919
17.810785804810262
36.44212213308962
34.81799960843838
349.36804030024564
34.71819336156373
357.57063200367696
17.046508511005495
357.2932778524239
279.13731369864206
279.21530509799777
278.18481844329375
357.621117937033
30.107538701032567
17

# junk

In [6]:
with open('Airbnb_Texas_Rentals_new.csv','r',encoding="utf8") as csv_file:
    csv_reader=csv.reader(csv_file)
    
    with open('Parametric_search.tsv','w',encoding="utf8") as new_file: 
        csv_writer=csv.writer(new_file,delimiter='\t')
    
        for line in csv_reader:
               csv_writer.writerow(line[2:5]) 

with open('Parametric_search.tsv','r',encoding="utf8") as all_files:
    for i, row in enumerate(all_files):
        if i > 0 and i < 40:
            if row != '\n':
                with open('doc_par_search\doc_' + str(int((i-(i/2)))) + '.tsv','w',encoding="utf8") as new_files:
                    new_files.write(row) 

In [15]:
docpaths = [f for f in listdir("doc_par_search") if isfile(join("doc_par_search", f))]

user_query = input('Enter city, price per night, number of bedrooms: ').split(' ')
city, price, bedrooms = user_query[0], user_query[1], user_query[2]



par_search_doc_list = {}
for doc in docpaths:
    with open('doc_par_search\{}'.format(doc), 'r',encoding="utf8") as file:
        lines = file.readlines()
        if len(lines) > 0 :
            document = lines[0]
            cols = document.replace('\n', '').split('\t')
            #doc_price, doc_bedrooms, doc_city = cols[1], cols[2], cols[3]
            #print(doc_price, doc_bedrooms, doc_city)
            par_search_doc_list[doc] = list(cols)
            
save_dict_to_file(par_search_doc_list,"par_search_doc_list")

Enter city, price per night, number of bedrooms: a b c


In [ ]:
#par_search_inv_idx = {}
#with open('par_search_doc_list.txt', 'r', encoding="utf-8") as file:
    data = eval(file.read())
    for key,doc in documentlist.items():
        for w in doc:
            score = tfidf(w,doc,documentlist)
            w_index = (key,score)
            if vocabulary[w] not in new_inverted_index:
                temp = []
                temp.append(w_index)
                new_inverted_index[vocabulary[w]] = temp
            elif doc not in new_inverted_index[vocabulary[w]]:
                new_inverted_index[vocabulary[w]].append(w_index)    